In [1]:
import pandas as pd

In [2]:
def read_jsonl(filename):
    df = pd.read_json(filename,lines=True)
    df = df.reset_index(drop=True)
    return df

In [3]:
import os

jsonl_directory = "/media/monlamai/SSD/staging/"

files = os.listdir(jsonl_directory)

pattern = "jsonl"
files_matching_pattern = [file for file in files if file.endswith(pattern)]

In [4]:
#all reviewed files 
reviewed = []
for file in files_matching_pattern:
    if 'review' in file and not 'stt_second_review' in file:
        reviewed.append(file)

In [5]:
#all unreviewed files
transcribed = []
for file in files_matching_pattern:
    if 'review' not in file and not 'stt_second_review' in file:
        transcribed.append(file)

In [6]:
#concatenate all the dataframes
temp= pd.DataFrame([])
for file in transcribed :
    df = read_jsonl(f"{jsonl_directory}/{file}")
    temp = pd.concat([temp,df],axis=0)
    trabscribed_df = temp.reset_index(drop=True)

In [7]:
#concatenate all the dataframes
temp= pd.DataFrame([])
for file in reviewed:
    df = read_jsonl(f"{jsonl_directory}/{file}")
    temp = pd.concat([temp,df],axis=0)
    reviewed_df = temp.reset_index(drop=True)

In [8]:
reviewed = temp

In [9]:
finale_df = read_jsonl(f"{jsonl_directory}/stt_second_review.jsonl")

In [10]:
trabscribed_df['id'] = trabscribed_df['id'].str.replace('.mp3','')
trabscribed_df['id'] = trabscribed_df['id'].str.replace('.wav','')
trabscribed_df['id'] = trabscribed_df['id'].str.replace('.MP3','')
trabscribed_df['id'] = trabscribed_df['id'].str.replace('.WAV','')

reviewed_df['id'] = reviewed_df['id'].str.replace('.mp3','')
reviewed_df['id'] = reviewed_df['id'].str.replace('.wav','')
reviewed_df['id'] = reviewed_df['id'].str.replace('.MP3','')
reviewed_df['id'] = reviewed_df['id'].str.replace('.WAV','')

finale_df['id'] = finale_df['id'].str.replace('.mp3','')
finale_df['id'] = finale_df['id'].str.replace('.wav','')
finale_df['id'] = finale_df['id'].str.replace('.MP3','')
finale_df['id'] = finale_df['id'].str.replace('.WAV','')

In [11]:
trabscribed_df['id'].fillna(trabscribed_df['text'], inplace=True)
reviewed_df['id'].fillna(reviewed_df['text'], inplace=True)
finale_df['id'].fillna(finale_df['text'], inplace=True)

In [12]:
len(trabscribed_df),len(reviewed_df),len(finale_df)

(515579, 468714, 200181)

In [13]:
len(set(trabscribed_df['id'])), len(set(reviewed_df['id'])), len(set(finale_df['id']))

(446421, 425269, 196295)

In [14]:
100 - len(set(trabscribed_df['id'])) / len(trabscribed_df) * 100

13.413657266878602

In [15]:
trabscribed_df.drop_duplicates(subset='id', keep="first", inplace=True)
reviewed_df.drop_duplicates(subset='id', keep="first", inplace=True)
finale_df.drop_duplicates(subset='id', keep="first", inplace=True)

In [16]:
len(trabscribed_df),len(reviewed_df),len(finale_df)

(446421, 425269, 196295)

In [17]:
trabscribed_df = trabscribed_df[trabscribed_df['answer'] == 'accept']
reviewed_df = reviewed_df[reviewed_df['answer'] == 'accept']
finale_df = finale_df[finale_df['answer'] == 'accept']

In [18]:
len(trabscribed_df),len(reviewed_df),len(finale_df)

(430387, 411073, 192520)

In [19]:
#left side intersection 
intersection = trabscribed_df.merge(reviewed_df, how='left', on='id')

In [20]:
intersection['transcript_y'].fillna(intersection['transcript_x'], inplace=True)


In [21]:
intersection = intersection[~intersection['transcript_y'].isna()]

In [22]:
len(intersection['id']), len(trabscribed_df['id'])

(429372, 430387)

In [23]:
last_intersection = intersection.merge(finale_df, how='left', on='id')

In [24]:
last_intersection.count()

id                   429372
audio_x              429372
url_x                 55899
transcript_x         428599
_input_hash_x        429372
_task_hash_x         429372
_view_id_x           429372
audio_spans_x        427742
answer_x             429372
_timestamp_x         423599
_annotator_id_x      429365
_session_id_x        429365
text_x               373473
meta_x               373473
path_x               373480
_is_binary_x         373480
field_rows_x             34
field_label_x            34
field_id_x               34
field_autofocus_x        34
audio_y              401961
text_y               365382
meta_y               363867
path_y               401940
_input_hash_y        401961
_task_hash_y         401961
_is_binary_y         401940
_view_id_y           401961
audio_spans_y        401960
transcript_y         429372
answer_y             401961
_timestamp_y         401944
_annotator_id_y      401961
_session_id_y        401961
field_rows_y         366069
field_label_y       

In [25]:
last_intersection['transcript'].fillna(last_intersection['transcript_y'], inplace=True)

In [26]:
last_intersection.shape

(429372, 59)

In [27]:
last_intersection = last_intersection[~last_intersection['transcript'].isna()]

In [28]:
last_intersection.shape

(429372, 59)

In [29]:
last_intersection = last_intersection[['transcript', 'id']]

In [30]:
last_intersection['url'] = 'https://d38pmlk0v88drf.cloudfront.net/wav/' + last_intersection['id'] + '.wav'

In [31]:
i = 110001
last_intersection.loc[i]['transcript'], last_intersection.loc[i,'url']

('ཨུ་ཨུན། ཨུ།',
 'https://d38pmlk0v88drf.cloudfront.net/wav/STT_TT00075_00080.000-00081.400.wav')

In [32]:
last_intersection['dept'] = last_intersection['id'].str[:6]

In [33]:
last_intersection.head()

,transcript,id,url,dept
0,དུང་དཀར་རིན་པོ་ཆེས་བླ་མ་དེར་ཕྱག་དཔེའི་ཆ་རྐྱེན་...,STT_AB00001_0041_209714_to_221712,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
1,དུང་དཀར་རིན་པོ་ཆེ,STT_AB00001_0042_222505_to_223855,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
2,རིན་པོ་ཆེ། ཁྱེད་ལ་སློབ་སྦྱོང་གི་ཆ་རྐྱེན་ཤིན་ཏ...,STT_AB00001_0043_224564_to_229795,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
3,ཅེས་གསུངས་པ་ན,STT_AB00001_0044_230453_to_231179,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
4,༄༅།། དུང་དཀར་རིན་པོ་ཆེ་ཆུང་དུས་ནས་བླ་མའི་ཡང་སྤ...,STT_AB00001_0031_154701_to_159021,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB


In [34]:
last_intersection.groupby('dept').count()

,transcript,id,url
dept,,,
STT_AB,55899,55899,55899
STT_CS,34629,34629,34629
STT_NS,45071,45071,45071
STT_TT,293773,293773,293773


In [35]:
import pyewts

converter = pyewts.pyewts()

last_intersection['wylie'] = last_intersection['transcript'].apply(converter.toWylie)

# བ་བྦ་a་བ་

In [36]:
last_intersection

,transcript,id,url,dept,wylie
0,དུང་དཀར་རིན་པོ་ཆེས་བླ་མ་དེར་ཕྱག་དཔེའི་ཆ་རྐྱེན་...,STT_AB00001_0041_209714_to_221712,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB,dung dkar rin po ches bla ma der phyag dpe'i c...
1,དུང་དཀར་རིན་པོ་ཆེ,STT_AB00001_0042_222505_to_223855,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB,dung dkar rin po che
2,རིན་པོ་ཆེ། ཁྱེད་ལ་སློབ་སྦྱོང་གི་ཆ་རྐྱེན་ཤིན་ཏ...,STT_AB00001_0043_224564_to_229795,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB,_rin po che/_khyed la slob sbyong gi cha rkyen...
3,ཅེས་གསུངས་པ་ན,STT_AB00001_0044_230453_to_231179,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB,ces gsungs pa na
4,༄༅།། དུང་དཀར་རིན་པོ་ཆེ་ཆུང་དུས་ནས་བླ་མའི་ཡང་སྤ...,STT_AB00001_0031_154701_to_159021,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB,@#//_dung dkar rin po che chung dus nas bla ma...
...,...,...,...,...,...
429367,དེ་དག་གི་དཀའ་ཇི་ལྟར་སྐྱེ་བ་,STT_TT00389_240.200-241.250,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,de dag gi dka' ji ltar skye ba
429368,མི་གཤིས་ཀ་དེ་འདྲའི་ཅན་གྱི་མི་ཅིག་ལ་རྟེན་ཅེ་,STT_TT00389_241.800-244.200,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,mi gshis ka de 'dra'i can gyi mi cig la rten ce
429369,ད་ངས་དེ་རིང་འདི་གྲོགས་པོ་བསྒྲིགས་བྱས་ང་ལ་སེམས་...,STT_TT00389_244.600-247.450,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,da ngas de ring 'di grogs po bsgrigs byas nga ...
429370,ང་ལ་ཁེ་བཟང་རག་ཡ་ཅིག་རེ་བ་བརྒྱབ་ཀི་,STT_TT00389_248.100-249.350,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,nga la khe bzang rag ya cig re ba brgyab ki


In [37]:
last_intersection.rename(columns={"id": "file_name", "transcript": "uni", "wylie": "wylie", "url": "url", "dept": "dept"}, inplace=True)

In [38]:
last_intersection = last_intersection[['file_name', 'uni', 'wylie', 'url', 'dept']]
last_intersection.head()

,file_name,uni,wylie,url,dept
0,STT_AB00001_0041_209714_to_221712,དུང་དཀར་རིན་པོ་ཆེས་བླ་མ་དེར་ཕྱག་དཔེའི་ཆ་རྐྱེན་...,dung dkar rin po ches bla ma der phyag dpe'i c...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
1,STT_AB00001_0042_222505_to_223855,དུང་དཀར་རིན་པོ་ཆེ,dung dkar rin po che,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
2,STT_AB00001_0043_224564_to_229795,རིན་པོ་ཆེ། ཁྱེད་ལ་སློབ་སྦྱོང་གི་ཆ་རྐྱེན་ཤིན་ཏ...,_rin po che/_khyed la slob sbyong gi cha rkyen...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
3,STT_AB00001_0044_230453_to_231179,ཅེས་གསུངས་པ་ན,ces gsungs pa na,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
4,STT_AB00001_0031_154701_to_159021,༄༅།། དུང་དཀར་རིན་པོ་ཆེ་ཆུང་དུས་ནས་བླ་མའི་ཡང་སྤ...,@#//_dung dkar rin po che chung dus nas bla ma...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB


In [39]:
last_intersection.to_csv('prodigy.tsv', sep='\t', index=False)

In [40]:
last_intersection[last_intersection['file_name'] == 'STT_AB00001_0029_150584_to_151748']

,file_name,uni,wylie,url,dept
16,STT_AB00001_0029_150584_to_151748,ལེའུ་དང་པོ།\n,_le'u dang po/\n,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_AB
